# Recommendation Engine : Surprise

https://surprise.readthedocs.io/en/stable/index.html

In [1]:
import pandas as pd
import numpy as np

### Automatic Cross-Validation

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [3]:
data = Dataset.load_builtin('ml-100k')

In [4]:
# Use the famous SVD algorithm
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9358  0.9351  0.9439  0.9299  0.9374  0.9364  0.0045  
MAE (testset)     0.7387  0.7367  0.7434  0.7327  0.7379  0.7379  0.0035  
Fit time          4.43    4.29    4.47    4.08    4.11    4.28    0.16    
Test time         0.11    0.17    0.10    0.16    0.17    0.14    0.03    


{'test_rmse': array([0.93578344, 0.9350695 , 0.94386   , 0.92993934, 0.93742027]),
 'test_mae': array([0.7387    , 0.73669761, 0.74343168, 0.73270502, 0.73792779]),
 'fit_time': (4.42892599105835,
  4.292311906814575,
  4.4747231006622314,
  4.082387924194336,
  4.107619762420654),
 'test_time': (0.11090302467346191,
  0.16754627227783203,
  0.09827494621276855,
  0.16051721572875977,
  0.16569018363952637)}

### Train-Test Split and Fit Method

In [5]:
from surprise.model_selection import train_test_split
from surprise import accuracy

# Split data by trainset and testset. Test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.20)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9346


0.9345567873481592

### Trainset and the Predict Methods

In [6]:
from surprise import KNNBasic
from surprise import Dataset

# Load the movielens-100k dataset
data = Dataset.load_builtin('ml-100k')

# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [7]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.06   {'actual_k': 40, 'was_impossible': False}


### Custom Dataset

#### Create a Pandas DataFrame

In [25]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import NormalPredictor


# Create dataframe. Column names are irrelevant.
ratings_dict = {
    'item': [1, 1, 1, 2, 2, 1, 3, 2],
    'user': [9, 32, 2, 45, 'foo',19, 51, 32],
    'rating': [3, 2, 4, 3, 1, 2, 2, 5]
}

df = pd.DataFrame(ratings_dict)

In [9]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df( df[['user', 'item', 'rating']], reader )

algo = NormalPredictor()
cross_validate( algo, data, measures=['rmse'], cv=2)

{'test_rmse': array([2.11701331, 2.03587607]),
 'fit_time': (9.107589721679688e-05, 4.410743713378906e-05),
 'test_time': (4.696846008300781e-05, 3.2901763916015625e-05)}

In [10]:
# Train the model
trainset = data.build_full_trainset()

algo = NormalPredictor()
algo.fit(trainset)

In [11]:
# Use trainset as testset

# print( trainset.build_anti_testset() )
print( trainset.build_testset() )

[(9, 1, 3.0), (32, 1, 2.0), (32, 2, 5.0), (2, 1, 4.0), (45, 2, 3.0), ('foo', 2, 1.0), (19, 1, 2.0), (51, 3, 2.0)]


Compute the rating prediction for a given user-item pair

In [27]:
for item in trainset.build_testset():
    uid, iid, r_ui = item
    res = algo.predict( uid, iid, r_ui=r_ui, verbose=True) 

user: 9          item: 1          r_ui = 3.00   est = 2.13   {'was_impossible': False}
user: 32         item: 1          r_ui = 2.00   est = 1.00   {'was_impossible': False}
user: 32         item: 2          r_ui = 5.00   est = 1.96   {'was_impossible': False}
user: 2          item: 1          r_ui = 4.00   est = 1.87   {'was_impossible': False}
user: 45         item: 2          r_ui = 3.00   est = 1.17   {'was_impossible': False}
user: foo        item: 2          r_ui = 1.00   est = 2.37   {'was_impossible': False}
user: 19         item: 1          r_ui = 2.00   est = 3.09   {'was_impossible': False}
user: 51         item: 3          r_ui = 2.00   est = 2.90   {'was_impossible': False}


In [28]:
print(res.uid, res.iid, res.r_ui, round(res.est,2) )

51 3 2.0 2.9


In [29]:
# Prediction
trainset = trainset.build_testset()

predictions = algo.test(trainset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.5139


1.5138683084415245

###  Example

In [45]:
import random                                                              

from surprise import SVD                                                   
from surprise import Dataset                                               
from surprise import accuracy                                              
from surprise.model_selection import GridSearchCV                                       


# Load your full dataset.                                                  
data = Dataset.load_builtin('ml-100k')       

# List of typles : (uid, iid, rt, ts)
raw_ratings = data.raw_ratings                                             

# Shuffle ratings if you want                                              
random.shuffle(raw_ratings)                                                

# 90% trainset, 10% testset                                                
threshold = int(0.9 * len(raw_ratings))                                     
train_raw_ratings = raw_ratings[:threshold]                             
test_raw_ratings  = raw_ratings[threshold:]                                 

In [52]:
train_raw_ratings[0:3]
test_raw_ratings[0:3]

[('417', '418', 4.0, '879647471'),
 ('387', '204', 2.0, '886479771'),
 ('458', '603', 4.0, '886397155')]

In [44]:
data.raw_ratings = train_raw_ratings  # data is now the trainset       

In [50]:
# Select your best algo with grid search. Verbosity is buggy, I'll fix it. 

print('GRID SEARCH...')                                                    
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005]}               
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)    
grid_search.fit(data)                                                 

algo = grid_search.best_estimator['rmse']                     

GRID SEARCH...


In [54]:
# retrain on the whole train set                                           
trainset = data.build_full_trainset()                                      
algo.fit(trainset)     

In [55]:
# test on the trainset                                                 
testset = data.construct_testset(train_raw_ratings)                     
predictions = algo.test(testset)                                           
print('Accuracy on the trainset:')                                         
accuracy.rmse(predictions)                                                 

# test on the testset                                                  
testset = data.construct_testset(test_raw_ratings)                         
predictions = algo.test(testset)                                           
print('Accuracy on the testset:')                                          
accuracy.rmse(predictions)  

Accuracy on the trainset:
RMSE: 0.8367
Accuracy on the testset:
RMSE: 0.8307


0.830656557608851

In [56]:
data.construct_testset(train_raw_ratings)

[('800', '276', 3.0),
 ('461', '255', 2.0),
 ('648', '722', 3.0),
 ('405', '542', 1.0),
 ('710', '501', 3.0),
 ('145', '315', 5.0),
 ('429', '233', 3.0),
 ('13', '89', 4.0),
 ('753', '523', 4.0),
 ('466', '210', 4.0),
 ('881', '826', 1.0),
 ('861', '584', 5.0),
 ('262', '111', 4.0),
 ('102', '771', 2.0),
 ('484', '239', 4.0),
 ('181', '148', 2.0),
 ('774', '195', 3.0),
 ('749', '154', 5.0),
 ('899', '161', 4.0),
 ('340', '428', 1.0),
 ('592', '289', 4.0),
 ('936', '7', 4.0),
 ('569', '328', 4.0),
 ('111', '321', 3.0),
 ('181', '1187', 1.0),
 ('387', '428', 4.0),
 ('606', '172', 5.0),
 ('855', '283', 3.0),
 ('707', '582', 5.0),
 ('416', '157', 4.0),
 ('398', '87', 4.0),
 ('474', '652', 4.0),
 ('151', '546', 2.0),
 ('606', '91', 5.0),
 ('918', '606', 4.0),
 ('815', '675', 2.0),
 ('790', '100', 2.0),
 ('394', '33', 4.0),
 ('658', '960', 4.0),
 ('101', '1034', 2.0),
 ('718', '282', 5.0),
 ('297', '515', 5.0),
 ('885', '167', 3.0),
 ('494', '100', 5.0),
 ('458', '387', 4.0),
 ('682', '12', 

In [58]:
data.construct_trainset(train_raw_ratings)